# PANDA EfficientNet-B0 Baseline with 36 x tiles_256

Hi everyone,

I'm here to show you how to train a single efficientnet-b0 model to get LB 0.87

This is inference kernel and the Training kernel is avalilable here: https://www.kaggle.com/haqishen/train-efficientnet-b0-w-36-tiles-256-lb0-87



# TTA
    
Tile extraction start from different point (by adding more white padding)

In [1]:
DEBUG = False

In [2]:
import os
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [3]:
import skimage.io
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import model as enet
from sklearn.metrics import cohen_kappa_score

import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm



Bad key "text.kerning_factor" on line 4 in
/data/zhaoxun/miniconda3/envs/torch/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [5]:
data_dir = '/home/zhaoxun/codes/Panda/_data'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_sub = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

model_dir = '/home/zhaoxun/codes/Panda/_models'
image_folder = os.path.join(data_dir, 'test_images')
is_test = os.path.exists(image_folder)  # IF test_images is not exists, we will use some train images.
image_folder = image_folder if is_test else os.path.join(data_dir, 'train_images')

df = df_test if is_test else df_train.loc[:100]

tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 2
num_workers = 4

device = torch.device('cuda')

print(image_folder)

/home/zhaoxun/codes/Panda/_data/train_images


# Model

In [6]:
def gem(x, p=3, eps=1e-6):
    return torch.nn.functional.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = torch.nn.parameter.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'


class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.enet._avg_pooling = GeM()
# 
    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x
    
    
def load_models(model_files):
    models = []
    for model_f, dim in model_files:
#         model_f = os.path.join(model_dir, model_f)
        backbone = 'efficientnet-b0'
        model = enetv2(backbone, out_dim = dim)
        model.load_state_dict(torch.load(model_f))
        model.eval()
        model.to(device)
        models.append(model)
        print(f'{model_f} loaded!')
    return models


model_files = [
#     '../input/panda-public-models/cls_effnet_b0_Rand36r36tiles256_big_bce_lr0.3_augx2_30epo_model_fold0.pth',
#     '../input/36256-fold0/fold0.my.model',
#     '../input/36256-fold0/fold1.my.model',
#     '../input/36256-fold0/fold2.my.model'
#     "../input/bs16models/fold%d.pth" % i for i in range(4, 5),
    ("/home/zhaoxun/codes/Panda/_models/Jun.15_15:44.cls.final.model", 5),
    ("/home/zhaoxun/codes/Panda/_models/Jun.15_23:28.cls.final.model", 6),
#     "../input/bs16models/fold0.1.1weight.pth",
#     "../input/bs16models/fold0.finetune.pth",
#     "../input/bs16models/fold0.biglr.over.pth",
]

models = load_models(model_files)

/home/zhaoxun/codes/Panda/_models/Jun.15_15:44.cls.final.model loaded!
/home/zhaoxun/codes/Panda/_models/Jun.15_23:28.cls.final.model loaded!


# Dataset

In [7]:
def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img3) < n_tiles:
            img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles


class PANDADataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 sub_imgs=False
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.sub_imgs = sub_imgs

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[1]
        tiles, OK = get_tiles(image, self.tile_mode)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))
        idxes = np.asarray(idxes) + self.n_tiles if self.sub_imgs else idxes

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img

#         images = 255 - images
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        return torch.tensor(images)


# Prediction

In [8]:
dataset = PANDADataset(df, image_size, n_tiles, 0)  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2)  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)

In [9]:
LOGITS = []
LOGITS2 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        LOGITS.append(models[0](data).sigmoid())
        for model in models[1:]:
            LOGITS[-1] += model(data).sigmoid()
        for tmp in [data.flip(-1), data.flip(-2), data.flip(-1, -2), data.transpose(-1,-2), data.transpose(-1,-2).flip(-1), data.transpose(-1,-2).flip(-2), data.transpose(-1,-2).flip(-1, -2)]:
            for model in models:
                LOGITS[-1] += model(tmp).sigmoid()

    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(models[0](data).sigmoid())
        for model in models[1:]:
            LOGITS2[-1] += model(data).sigmoid()
        for tmp in [data.flip(-1), data.flip(-2), data.flip(-1, -2), data.transpose(-1,-2), data.transpose(-1,-2).flip(-1), data.transpose(-1,-2).flip(-2), data.transpose(-1,-2).flip(-1, -2)]:
            for model in models:
                LOGITS2[-1] += model(data).sigmoid()

# LOGITS2 = LOGITS
LOGITS = (torch.cat(LOGITS).cpu() + torch.cat(LOGITS2).cpu()) / (2 * len(models) * 8)
PREDS = LOGITS.sum(1).round().numpy()

target = df.isup_grade
df['isup_grade'] = PREDS.astype(int)
df[['image_id', 'isup_grade']].to_csv('submission.csv', index=False)
print(df)
print(df.isup_grade.value_counts())
print(cohen_kappa_score(target, df.isup_grade))

0%|          | 0/51 [00:03<?, ?it/s]


RuntimeError: The size of tensor a (5) must match the size of tensor b (6) at non-singleton dimension 1

In [21]:
with torch.no_grad():
    for data in loader:
        break
    print(data[2,0])
    print(models[0](data[2:3].cuda()))
    print(models[0](data[2:3].cuda()).sigmoid().sum())

tensor([[0.1137, 0.1843, 0.1490,  ..., 0.1529, 0.1608, 0.1608],
        [0.1569, 0.1412, 0.0863,  ..., 0.1608, 0.1647, 0.1569],
        [0.1216, 0.1294, 0.1216,  ..., 0.1490, 0.1490, 0.1412],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0510, 0.0510, 0.0510],
        [0.0000, 0.0000, 0.0000,  ..., 0.0510, 0.0510, 0.0510],
        [0.0000, 0.0000, 0.0000,  ..., 0.0510, 0.0510, 0.0510]])
tensor([[  9.0012,  19.4930,  15.1528,   9.4424, -10.0495]], device='cuda:0')
tensor(3.9998, device='cuda:0')


In [24]:
for p in list(models[0].parameters())[-1:]:
    break
print(p)

Parameter containing:
tensor([-0.0184, -0.0057, -0.0305, -0.0100, -0.0429], device='cuda:0',
       requires_grad=True)


In [25]:
import efficientnet_pytorch
print(efficientnet_pytorch.__version__)

0.6.3
